## <b><font color='darkblue'>Preface</font></b>
<font size='3ptx'><b>In the [Agent Development Kit](https://google.github.io/adk-docs/agents/) (ADK), an Agent is a self-contained execution unit designed to act autonomously to achieve specific goals.</b> Agents can perform tasks, interact with users, utilize external tools, and coordinate with other agents.</font> 

<b>The foundation for all agents in ADK is the <font color='blue'>BaseAgent</font> class.</b> It serves as the fundamental blueprint. To create functional agents, you typically extend <font color='blue'>**BaseAgent**</font> in one of three main ways, catering to different needs – from intelligent reasoning to structured process control:
![agent types](https://google.github.io/adk-docs/assets/agent-types.png)

In [2]:
from IPython.display import display, Markdown, Latex

def show_source_code(src_path: str):
    source_code = !cat $src_path
    display(Markdown(f"""
```python
{'\n'.join(source_code)}
```"""))

### <b><font color='darkgreen'>Core Agent Categories</font></b>
([source](https://google.github.io/adk-docs/agents/#core-agent-categories)) ADK provides distinct agent categories to build sophisticated applications:
1. [**LLM Agents (LlmAgent, Agent)**](https://google.github.io/adk-docs/agents/llm-agents/): These agents utilize Large Language Models (LLMs) as their core engine to understand natural language, reason, plan, generate responses, and dynamically decide how to proceed or which tools to use, making them ideal for flexible, language-centric tasks. [Learn more about LLM Agents...](https://google.github.io/adk-docs/agents/llm-agents/)
2. [**Workflow Agents (SequentialAgent, ParallelAgent, LoopAgent)**](https://google.github.io/adk-docs/agents/workflow-agents/): These specialized agents control the execution flow of other agents in predefined, deterministic patterns (sequence, parallel, or loop) without using an LLM for the flow control itself, perfect for structured processes needing predictable execution. [Explore Workflow Agents...](https://google.github.io/adk-docs/agents/workflow-agents/)
3. [**Custom Agents**](https://google.github.io/adk-docs/agents/custom-agents/): Created by extending <font color='blue'>**BaseAgent**</font> directly, these agents allow you to implement unique operational logic, specific control flows, or specialized integrations not covered by the standard types, catering to highly tailored application requirements. [Discover how to build Custom Agents...](https://google.github.io/adk-docs/agents/custom-agents/)

### <b><font color='darkgreen'>Choosing the Right Agent Type</font></b>
<b>The following table provides a high-level comparison to help distinguish between the agent types</b>. As you explore each type in more detail in the subsequent sections, these distinctions will become clearer.

| Feature           | LLM Agent (`LlmAgent`)                   | Workflow Agent                        | Custom Agent (`BaseAgent` subclass)           |
| :---------------- | :-------------------------------------- | :------------------------------------ | :-------------------------------------------- |
| **Primary Function** | Reasoning, Generation, Tool Use         | Controlling Agent Execution Flow      | Implementing Unique Logic/Integrations        |
| **Core Engine** | Large Language Model (LLM)              | Predefined Logic (Sequence, Parallel, Loop) | Custom Code                                   |
| **Determinism** | Non-deterministic (Flexible)            | Deterministic (Predictable)           | Can be either, based on implementation        |
| **Primary Use** | Language tasks, Dynamic decisions       | Structured processes, Orchestration   | Tailored requirements, Specific workflows     |

### <b><font color='darkgreen'>Agents Working Together: Multi-Agent Systems</font></b>
<b><font size='3ptx'>While each agent type serves a distinct purpose, the true power often comes from combining them</font></b>. Complex applications frequently employ multi-agent architectures where:
* **LLM Agents** handle intelligent, language-based task execution.
* **Workflow Agents** manage the overall process flow using standard patterns.
* **Custom Agents** provide specialized capabilities or rules needed for unique integrations.

Understanding these core types is the first step toward building sophisticated, capable AI applications with ADK.

### <b><font color='darkgreen'>What's Next?</font></b>
Now that you have an overview of the different agent types available in ADK, dive deeper into how they work and how to use them effectively:

* [**LLM Agents**](https://google.github.io/adk-docs/agents/llm-agents/): Explore how to configure agents powered by large language models, including setting instructions, providing tools, and enabling advanced features like planning and code execution.
* [**Workflow Agents**](https://google.github.io/adk-docs/agents/workflow-agents/): Learn how to orchestrate tasks using `SequentialAgent`, `ParallelAgent`, and `LoopAgent` for structured and predictable processes.
* [**Custom Agents**](https://google.github.io/adk-docs/agents/custom-agents/): Discover the principles of extending `BaseAgent` to build agents with unique logic and integrations tailored to your specific needs.
* [**Multi-Agents**](https://google.github.io/adk-docs/agents/multi-agents/): Understand how to combine different agent types to create sophisticated, collaborative systems capable of tackling complex problems.
* [**Models**](https://google.github.io/adk-docs/agents/models/): Learn about the different LLM integrations available and how to select the right model for your agents.

## <b><font color='darkblue'>LLM Agent</font></b>
<font size='3ptx'><b>The <font color='blue'>LlmAgent</font> (<font color='brown'>often aliased simply as `Agent`</font>) is a core component in ADK, acting as the "thinking" part of your application.</b> It leverages the power of a Large Language Model (LLM) for reasoning, understanding natural language, making decisions, generating responses, and interacting with tools.</font>

<b>Unlike deterministic `Workflow Agents` that follow predefined execution paths, `LlmAgent` behavior is non-deterministic</b>. It uses the LLM to interpret instructions and context, deciding dynamically how to proceed, which tools to use (<font color='brown'>if any</font>), or whether to transfer control to another agent.

Building an effective <font color='blue'>**LlmAgent**</font> involves defining its identity, clearly guiding its behavior through instructions, and equipping it with the necessary tools and capabilities.

### <b><font color='darkgreen'>Defining the Agent's Identity and Purpose</font></b>
First, you need to establish what the agent is and what it's for.

* **name (Required)**: Every agent needs a unique string identifier. This `name` is crucial for internal operations, especially in multi-agent systems where agents need to refer to or delegate tasks to each other. Choose a descriptive name that reflects the agent's function (<font color='brown'>e.g., `customer_support_router`, `billing_inquiry_agent`). Avoid reserved names like `user`.
* **description (Optional, Recommended for Multi-Agent)**: Provide a concise summary of the agent's capabilities. This description is primarily used by other LLM agents to determine if they should route a task to this agent. Make it specific enough to differentiate it from peers (<font color='brown'>e.g., "Handles inquiries about current billing statements," not just "Billing agent"</font>).
* **model (Required)**: Specify the underlying LLM that will power this agent's reasoning. This is a string identifier like "`gemini-2.0-flash`". The choice of model impacts the agent's capabilities, cost, and performance. See the [Models page](https://google.github.io/adk-docs/agents/models/) for available options and considerations.

```python
# Example: Defining the basic identity
capital_agent = LlmAgent(
    model="gemini-2.0-flash",
    name="capital_agent",
    description="Answers user questions about the capital city of a given country."
    # instruction and tools will be added next
)
```

### <b><font color='darkgreen'>Guiding the Agent: Instructions (`instruction`)</font></b>
The `instruction` parameter is arguably the most critical for shaping an <b><font color='blue'>LlmAgent</font></b>'s behavior. It's a string (<font color='brown'>or a function returning a string</font>) that tells the agent:
* Its core task or goal.
* Its personality or persona (<font color='brown'>e.g., "You are a helpful assistant," "You are a witty pirate"</font>).
* Constraints on its behavior (<font color='brown'>e.g., "Only answer questions about X," "Never reveal Y"</font>).
* How and when to use its tools. You should explain the purpose of each tool and the circumstances under which it should be called, supplementing any descriptions within the tool itself.
* The desired format for its output (<font color='brown'>e.g., "Respond in JSON," "Provide a bulleted list"</font>).

Tips for Effective Instructions:
* **Be Clear and Specific**: Avoid ambiguity. Clearly state the desired actions and outcomes.
* **Use Markdown**: Improve readability for complex instructions using headings, lists, etc.
* **Provide Examples (Few-Shot)**: For complex tasks or specific output formats, include examples directly in the instruction.
* **Guide Tool Use**: Don't just list tools; explain when and why the agent should use them.

State:
* The instruction is a string template, you can use the `{var}` syntax to insert dynamic values into the instruction.
* `{var}` is used to insert the value of the state variable named `var`.
* `{artifact.var}` is used to insert the text content of the `artifact` named `var`.
* **If the state variable or artifact does not exist, the agent will raise an error**. If you want to ignore the error, you can append a `?` to the variable name as in `{var?}`.

```python
# Example: Adding instructions
capital_agent = LlmAgent(
    model="gemini-2.0-flash",
    name="capital_agent",
    description="Answers user questions about the capital city of a given country.",
    instruction="""You are an agent that provides the capital city of a country.
When a user asks for the capital of a country:
1. Identify the country name from the user's query.
2. Use the `get_capital_city` tool to find the capital.
3. Respond clearly to the user, stating the capital city.
Example Query: "What's the capital of {country}?"
Example Response: "The capital of France is Paris."
""",
    # tools will be added next
)
```

<b><font color='darkred'>Note:</font></b> For instructions that apply to all agents in a system, consider using `global_instruction` on the root agent, detailed further in the [**Multi-Agents section**](https://google.github.io/adk-docs/agents/multi-agents/).

### <b><font color='darkgreen'>Equipping the Agent: Tools (`tools`)</font></b>
<font size='3ptx'><b>Tools give your <font color='blue'>LlmAgent</font> capabilities beyond the LLM's built-in knowledge or reasoning</b></font>. They allow the agent to interact with the outside world, perform calculations, fetch real-time data, or execute specific actions.
* **tools (Optional)**: Provide a list of tools the agent can use. Each item in the list can be:
  - **A native function or method (wrapped as a `FunctionTool`)**. Python ADK automatically wraps the native function into a <font color='blue'>**FuntionTool**</font>.
  - An instance of a class inheriting from <b><font color='blue'>BaseTool</font></b>.
  - An instance of another agent (<font color='brown'>`AgentTool`, enabling agent-to-agent delegation - see [Multi-Agents](https://google.github.io/adk-docs/agents/multi-agents/)</font>).

The LLM uses the function/tool names, descriptions (<font color='brown'>from docstrings or the `description` field</font>), and parameter schemas to decide which tool to call based on the conversation and its instructions.

```python
# Define a tool function
def get_capital_city(country: str) -> str:
  """Retrieves the capital city for a given country."""
  # Replace with actual logic (e.g., API call, database lookup)
  capitals = {"france": "Paris", "japan": "Tokyo", "canada": "Ottawa"}
  return capitals.get(country.lower(), f"Sorry, I don't know the capital of {country}.")

# Add the tool to the agent
capital_agent = LlmAgent(
    model="gemini-2.0-flash",
    name="capital_agent",
    description="Answers user questions about the capital city of a given country.",
    instruction="""You are an agent that provides the capital city of a country... (previous instruction text)""",
    tools=[get_capital_city] # Provide the function directly
)
```

Learn more about Tools in the [**Tools section**](https://google.github.io/adk-docs/tools/).

### <b><font color='darkgreen'>Advanced Configuration & Control</font></b>
([source](https://google.github.io/adk-docs/agents/llm-agents/#advanced-configuration-control)) Beyond the core parameters, <font color='blue'>**LlmAgent**</font> offers several options for finer control:

#### <b>Fine-Tuning LLM Generation (`generate_content_config`)</b>
You can adjust how the underlying LLM generates responses using `generate_content_config`.
* **generate_content_config (Optional)**: Pass an instance of <b><font color='blue'>google.genai.types.GenerateContentConfig</font></b> to control parameters like `temperature` (randomness), `max_output_tokens` (response length), `top_p`, `top_k`, and `safety` settings.

```python
from google.genai import types

agent = LlmAgent(
    # ... other params
    generate_content_config=types.GenerateContentConfig(
        temperature=0.2, # More deterministic output
        max_output_tokens=250
    )
)
```

#### <b>Structuring Data (`input_schema`, `output_schema`, `output_key`)</b>
For scenarios requiring structured data exchange with an LLM Agent, the ADK provides mechanisms to <b>define expected input and desired output formats using `schema` definitions</b>.

* **input_schema (Optional)**: Define a schema representing the expected input structure. If set, the user message content passed to this agent must be a JSON string conforming to this schema. Your instructions should guide the user or preceding agent accordingly.
* **output_schema (Optional)**: Define a schema representing the desired output structure. If set, the agent's final response must be a JSON string conforming to this schema.
  - **Constraint**: Using `output_schema` enables controlled generation within the LLM but **disables the agent's ability to use tools or transfer control to other agents**. Your instructions must guide the LLM to produce JSON matching the schema directly.
* **output_key (Optional)**: Provide a string key. If set, the text content of the **agent's final response will be automatically saved to the session's state dictionary under this key**. This is useful for passing results between agents or steps in a workflow. This might look like: `session.state[output_key] = agent_response_text`.

The input and output schema is typically a [**Pydantic BaseModel**](https://docs.pydantic.dev/latest/api/base_model/).

```python
from pydantic import BaseModel, Field

class CapitalOutput(BaseModel):
    capital: str = Field(description="The capital of the country.")

structured_capital_agent = LlmAgent(
    # ... name, model, description
    instruction="""You are a Capital Information Agent. Given a country, respond ONLY with a JSON object containing the capital. Format: {"capital": "capital_name"}""",
    output_schema=CapitalOutput, # Enforce JSON output
    output_key="found_capital"  # Store result in state['found_capital']
    # Cannot use tools=[get_capital_city] effectively here
)
```

#### <b>Managing Context (`include_contents`)</font>
Control whether the agent receives the prior conversation history.
* **include_contents (Optional, Default: '`default`')**: Determines if the `contents` (history) are sent to the LLM.
  - `'default'`: The agent receives the relevant conversation history.
  - `'none'`: The agent receives no prior contents. It operates based solely on its current instruction and any input provided in the current turn (<font color='brown'>useful for stateless tasks or enforcing specific contexts</font>).

```python
stateless_agent = LlmAgent(
    # ... other params
    include_contents='none'
)
```

#### <b>Planner</b>
**planner (Optional):** Assign a <font color='blue'>**BasePlanner**</font> instance to enable multi-step reasoning and planning before execution. There are two main planners:
* **BuiltInPlanner**: Leverages the model's built-in planning capabilities (<font color='brown'>e.g., Gemini's thinking feature</font>). See [Gemini Thinking](https://ai.google.dev/gemini-api/docs/thinking) for details and examples.

Here, the <font color='violet'>thinking_budget</font> parameter guides the model on the number of thinking tokens to use when generating a response. The <font color='violet'>include_thoughts</font> parameter controls whether the model should include its raw thoughts and internal reasoning process in the response.

```python
from google.adk import Agent
from google.adk.planners import BuiltInPlanner
from google.genai import types

my_agent = Agent(
    model="gemini-2.5-flash",
    planner=BuiltInPlanner(
        thinking_config=types.ThinkingConfig(
            include_thoughts=True,
            thinking_budget=1024,
        )
    ),
    # ... your tools here
)
```

* **PlanReActPlanner:** This planner instructs the model to follow a specific structure in its output: first create a plan, then execute actions (<font color='brown'>like calling tools</font>), and provide reasoning for its steps. It's particularly useful for models that don't have a `built-in "thinking"` feature.

```python
from google.adk import Agent
from google.adk.planners import PlanReActPlanner

my_agent = Agent(
    model="gemini-2.0-flash",
    planner=PlanReActPlanner(),
    # ... your tools here
)
```

The agent's response will follow a structured format:
```
[user]: ai news
[google_search_agent]: /*PLANNING*/
1. Perform a Google search for "latest AI news" to get current updates and headlines related to artificial intelligence.
2. Synthesize the information from the search results to provide a summary of recent AI news.

/*ACTION*/
/*REASONING*/
The search results provide a comprehensive overview of recent AI news, covering various aspects like company developments, research breakthroughs, and applications. I have enough information to answer the user's request.

/*FINAL_ANSWER*/
Here's a summary of recent AI news:
....
```

#### <b>Code Execution</b>
**code_executor (Optional)**: Provide a <b><font color='blue'>BaseCodeExecutor</font></b> instance to allow the agent to execute code blocks found in the LLM's response. ([See Tools/Built-in tools](https://google.github.io/adk-docs/tools/built-in-tools/)).

### <b><font color='darkgreen'>Putting It Together: Example</font></b>
Let's check the example `agent_example1.py` as below:

In [8]:
show_source_code('agent_example1.py')


```python
#!/usr/bin/env python
# --- Full example code demonstrating LlmAgent with Tools vs. Output Schema ---
import asyncio
import json # Needed for pretty printing dicts

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from pydantic import BaseModel, Field

# --- 1. Define Constants ---
APP_NAME = "agent_comparison_app"
USER_ID = "test_user_456"
SESSION_ID_TOOL_AGENT = "session_tool_agent_xyz"
SESSION_ID_SCHEMA_AGENT = "session_schema_agent_xyz"
MODEL_NAME = "gemini-2.0-flash"

# --- 2. Define Schemas ---

# Input schema used by both agents
class CountryInput(BaseModel):
  country: str = Field(description="The country to get information about.")

# Output schema ONLY for the second agent
class CapitalInfoOutput(BaseModel):
  capital: str = Field(description="The capital city of the country.")
  # Note: Population is illustrative; the LLM will infer or estimate this
  # as it cannot use tools when output_schema is set.
  population_estimate: str = Field(
      description="An estimated population of the capital city.")

# --- 3. Define the Tool (Only for the first agent) ---
def get_capital_city(country: str) -> str:
  """Retrieves the capital city of a given country."""
  print(f"\n-- Tool Call: get_capital_city(country='{country}') --")
  country_capitals = {
      "united states": "Washington, D.C.",
      "canada": "Ottawa",
      "france": "Paris",
      "japan": "Tokyo",
      "taiwan": "Taipei",
  }
  result = country_capitals.get(
      country.lower(), f"Sorry, I couldn't find the capital for {country}.")
  print(f"-- Tool Result: '{result}' --")
  return result

# --- 4. Configure Agents ---

# Agent 1: Uses a tool and output_key
capital_agent_with_tool = LlmAgent(
    model=MODEL_NAME,
    name="capital_agent_tool",
    description="Retrieves the capital city using a specific tool.",
    instruction="""
You are a helpful agent to provide the capital city of a country using a tool.
The user will provide the country name in a JSON format like {"country": "country_name"}.

1. Extract the country name.
2. Use the `get_capital_city` tool to find the capital.
3. Respond clearly to the user, stating the capital city found by the tool.
""",
    tools=[get_capital_city],
    input_schema=CountryInput,
    output_key="capital_tool_result", # Store final text response
)

# Agent 2: Uses output_schema (NO tools possible)
structured_info_agent_schema = LlmAgent(
    model=MODEL_NAME,
    name="structured_info_agent_schema",
    description=(
        "Provides capital and estimated population in a specific JSON format."),
    instruction=f"""
You are an agent that provides country information.
The user will provide the country name in a JSON format like
{{"country": "country_name"}}.

Respond ONLY with a JSON object matching this exact schema:
{json.dumps(CapitalInfoOutput.model_json_schema(), indent=2)}
Use your knowledge to determine the capital and estimate the population.

Do not use any tools.
""",
    # *** NO tools parameter here - using output_schema prevents tool use ***
    input_schema=CountryInput,
    output_schema=CapitalInfoOutput, # Enforce JSON output structure
    output_key="structured_info_result", # Store final JSON response
)


# --- 6. Define Agent Interaction Logic ---
async def call_agent_and_print(
    runner_instance: Runner,
    agent_instance: LlmAgent,
    session_service: InMemorySessionService,
    session_id: str,
    query_json: str,
):
  """Sends a query to the specified agent/runner and prints results."""
  print(f"\n>>> Calling Agent: '{agent_instance.name}' | Query: {query_json}")

  user_content = types.Content(
      role='user', parts=[types.Part(text=query_json)])

  final_response_content = "No final response received."
  async for event in runner_instance.run_async(
      user_id=USER_ID,
      session_id=session_id,
      new_message=user_content):
    # print(f"Event: {event.type}, Author: {event.author}") # Uncomment for detailed logging
    if event.is_final_response() and event.content and event.content.parts:
      # For output_schema, the content is the JSON string itself
      final_response_content = event.content.parts[0].text

  print(f"<<< Agent '{agent_instance.name}' Response: {final_response_content}")

  current_session = await session_service.get_session(
      app_name=APP_NAME,
      user_id=USER_ID,
      session_id=session_id)

  stored_output = current_session.state.get(agent_instance.output_key)

  # Pretty print if the stored output looks like JSON (likely from output_schema)
  print(f"--- Session State ['{agent_instance.output_key}']: ", end="")
  try:
    # Attempt to parse and pretty print if it's JSON
    parsed_output = json.loads(stored_output)
    print(json.dumps(parsed_output, indent=2))
  except (json.JSONDecodeError, TypeError):
    # Otherwise, print as string
    print(stored_output)
  print("-" * 30)


# --- 7. Run Interactions ---
async def main(capital_runner, structured_runner, session_service):
  print("--- Testing Agent with Tool ---")
  await call_agent_and_print(
      capital_runner,
      capital_agent_with_tool,
      session_service,
      SESSION_ID_TOOL_AGENT,
      '{"country": "Taiwan"}')

  await call_agent_and_print(
      capital_runner,
      capital_agent_with_tool,
      session_service,
      SESSION_ID_TOOL_AGENT,
      '{"country": "Canada"}')

  print("\n\n--- Testing Agent with Output Schema (No Tool Use) ---")
  await call_agent_and_print(
      structured_runner,
      structured_info_agent_schema,
      session_service,
      SESSION_ID_SCHEMA_AGENT,
      '{"country": "France"}')
  await call_agent_and_print(
      structured_runner,
      structured_info_agent_schema,
      session_service,
      SESSION_ID_SCHEMA_AGENT,
      '{"country": "Japan"}')


def create_runners():
  # --- 5. Set up Session Management and Runners ---
  session_service = InMemorySessionService()
  print(f'session_service: {session_service.__class__}')

  # Create separate sessions for clarity, though not strictly necessary if context is managed
  asyncio.run(session_service.create_session(
      app_name=APP_NAME,
      user_id=USER_ID,
      session_id=SESSION_ID_TOOL_AGENT))

  asyncio.run(session_service.create_session(
      app_name=APP_NAME,
      user_id=USER_ID,
      session_id=SESSION_ID_SCHEMA_AGENT))

  # Create a runner for EACH agent
  capital_runner = Runner(
      agent=capital_agent_with_tool,
      app_name=APP_NAME,
      session_service=session_service
  )
  structured_runner = Runner(
      agent=structured_info_agent_schema,
      app_name=APP_NAME,
      session_service=session_service
  )
  return capital_runner, structured_runner, session_service


if __name__ == "__main__":
  capital_runner, structured_runner, session_service = create_runners()
  asyncio.run(main(capital_runner, structured_runner, session_service))
```

Let's testing the agent examples below:

In [21]:
import agent_example1
from google.adk.runners import Runner
from google.genai import types


# --- 1. Define Constants ---
APP_NAME = "agent_comparison_app"
USER_ID = "test_user_456"
SESSION_ID="1234"

# --- 2. Define used class and instances
InMemorySessionService = agent_example1.InMemorySessionService
capital_agent_with_tool = agent_example1.capital_agent_with_tool 

In [12]:
capital_agent_with_tool

LlmAgent(name='capital_agent_tool', description='Retrieves the capital city using a specific tool.', parent_agent=None, sub_agents=[], before_agent_callback=None, after_agent_callback=None, model='gemini-2.0-flash', instruction='\nYou are a helpful agent to provide the capital city of a country using a tool.\nThe user will provide the country name in a JSON format like {"country": "country_name"}.\n\n1. Extract the country name.\n2. Use the `get_capital_city` tool to find the capital.\n3. Respond clearly to the user, stating the capital city found by the tool.\n', global_instruction='', tools=[<function get_capital_city at 0x7f69eedbfa60>], generate_content_config=None, disallow_transfer_to_parent=False, disallow_transfer_to_peers=False, include_contents='default', input_schema=<class 'agent_example1.CountryInput'>, output_schema=None, output_key='capital_tool_result', planner=None, code_executor=None, before_model_callback=None, after_model_callback=None, before_tool_callback=None, af

In [19]:
session_service = InMemorySessionService()
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=capital_agent_with_tool, app_name=APP_NAME, session_service=session_service)

In [24]:
# Call the agent
query = '{"country": "Taiwan"}'
content = types.Content(role='user', parts=[types.Part(text=query)])
events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

In [25]:
async for event in events:
    if event.is_final_response():
        final_response = event.content.parts[0].text
        print("Agent Response: ", final_response)


-- Tool Call: get_capital_city(country='Taiwan') --
-- Tool Result: 'Taipei' --
Agent Response:  The capital city of Taiwan is Taipei.



### <b><font color='darkgreen'>Related Concepts (Deferred Topics)</font></b>
While this page covers the core configuration of `LlmAgent`, several related concepts provide more advanced control and are detailed elsewhere:
* **Callbacks: Intercepting execution points** (<font color='brown'>before/after model calls, before/after tool calls</font>) using `before_model_callback`, `after_model_callback`, etc. See [**Callbacks**](https://google.github.io/adk-docs/callbacks/types-of-callbacks/).
* **Multi-Agent Control:** Advanced strategies for agent interaction, including planning (planner), controlling agent transfer (`disallow_transfer_to_parent`, `disallow_transfer_to_peers`), and system-wide instructions (`global_instruction`). See [**Multi-Agents**](https://google.github.io/adk-docs/agents/multi-agents/).